In [1]:
from collections import Counter
import math
from operator import itemgetter


def triplets(s: bytes):
    s = b' ' + s.strip() + b' '
    memview = memoryview(s)
    for i in range(len(s)-2):
        yield memview[i:i+3]
        

def all_triplets_from_many_lines(lines):
    for line in lines:
        yield from triplets(line)
        

def load_file(filename):
    with open(filename, 'rb') as file:
        return {line.rstrip(b'\r\n'): i for i, line in enumerate(file.readlines())}
    
    
def account_triplets(lines):
    c = Counter(all_triplets_from_many_lines(lines))
    m = max(c.values())
    for key in c:
        c[key] /= m  # Нормализуем по максимальному значению
    return c


def get_score(s: bytes):
    # return sum(c[t] for t in triplets(s)) / len(s)
    return sum(c[t] for t in triplets(s)) / math.log(len(s)+1)  # Преимущество длинным строкам

In [3]:
old_file = load_file('../stringdumps/stringdump_0_47_04.txt')
old_file.update(load_file('../stringdumps/stringdump_0_47_05.txt'))
old_file.update(load_file('../stringdumps/stringdump_0_47_03.txt'))
old_file.update(load_file('../stringdumps/stringdump_0_47_02.txt'))
old_file.update(load_file('../stringdumps/stringdump_0_47_01.txt'))
old_file.update(load_file('../stringdumps/stringdump_0_44_12.txt'))
old_file.update(load_file('../stringdumps/stringdump_steam_50_01.txt'))
old_file.update(load_file('../stringdumps/stringdump_steam_50_02.txt'))
old_file.update(load_file('../stringdumps/stringdump_steam_50_05.txt'))
old_file.update(load_file('../stringdumps/stringdump_steam_50_06.txt'))
old_file.update(load_file('../stringdumps/stringdump_steam_50_08.txt'))

c = account_triplets(old_file)  # Обучаем на старых файлах

In [5]:
new_file = load_file('../dfrus64/stringdump.txt')

In [6]:
diff = sorted(new_file.keys()-old_file.keys(), key=lambda s: get_score(s))

In [7]:
for item in diff:
    print(item, get_score(item))

b'\xe2\x94\x82d(' 0.0
b'!\xe2\x95\xa9B' 0.0
b'\xc3\xbct\xe2\x95\x92+\xe2\x89\xa1\xe2\x95\xa1' 0.0
b'6\xc3\xbbP' 0.0
b'l\xe2\x89\xa1\xe2\x95\xa1' 0.0
b'Pq-' 0.0
b'A\xe2\x96\xa0B\xe2\x89\xa1\xe2\x95\xa1' 0.0
b'P!\xc3\xbb' 0.0
b'A\xc2\xab\xc2\xac\xe2\x89\xa1\xe2\x95\xa1' 0.0
b'\xc3\xb1\\Y' 0.0
b'v\xe2\x88\x99\xc2\xb2' 0.0
b'-F' 0.0
b'\xe2\x95\xaaG\xc3\xbc' 0.0
b'\xe2\x94\x94\xe2\x94\x98M' 0.0
b'\xe2\x8c\xa1d\xe2\x89\xa1\xe2\x95\xa1' 0.0
b'a\xe2\x89\xa1\xe2\x95\xa1' 0.0
b'X\xc3\xb4\xc3\xb7' 0.0
b'lT\xe2\x95\x93' 0.0
b'\xe2\x8c\xa1Ma' 0.0
b'\xe2\x95\x94+B' 0.0
b'=\xce\xb5v' 0.0
b'.vZ' 0.0
b'A\xe2\x96\x8cUU \xc3\xa8\xe2\x82\xa7' 0.0
b'\xe2\x95\x97\xce\x98C' 0.0
b'\xce\xa6M' 0.0
b'R\xce\x93\xe2\x89\xa1\xe2\x95\xa1' 0.0
b"'dh" 0.0
b'k' 0.0
b';B\xe2\x89\xa1\xe2\x95\xa1' 0.0
b'uf\xe2\x89\xa1\xe2\x95\xa1' 0.0
b'\xe2\x94\x94F' 0.0
b'\xc3\xbaeu\xe2\x89\xa1\xe2\x95\xa1' 0.0
b'\xe2\x96\x91j' 0.0
b'\xe2\x96\x88\xe2\x94\x98a' 0.0
b'P\xe2\x95\xa3\xc3\x96' 0.0
b'z\xe2\x89\xa1\xe2\x95\xa1' 0.0
b"'9Y" 0.0


In [8]:
threshold = 0.01

In [10]:
with open('../stringdumps/stringdump_steam_50_09.txt', 'wb') as output:
    for line, number in sorted(new_file.items(), key=itemgetter(1)):
        if line in diff and get_score(line) < threshold: # Отсеиваем только добавившиеся строки
            continue
        
        output.write(line)
        output.write(b'\n')